In [0]:
import re

In [0]:
#temp
!python --version

Python 3.8.8

In [0]:
# Q1
from pyspark import SparkContext
sc = SparkContext()

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-811816059906892> in <module> 
 1 # Q1 
 2 from pyspark import SparkContext
 ----> 3 sc = SparkContext ( ) 

 /databricks/spark/python/pyspark/context.py in __init__ (self, master, appName, sparkHome, pyFiles, environment, batchSize, serializer, conf, gateway, jsc, profiler_cls) 
 143 " is not allowed as it is a security risk.")
 144 
 --> 145 SparkContext . _ensure_initialized ( self , gateway = gateway , conf = conf ) 
 146 try : 
 147 self._do_init(master, appName, sparkHome, pyFiles, environment, batchSize, serializer,

 /databricks/spark/python/pyspark/context.py in _ensure_initialized (cls, instance, gateway, conf) 
 348 
 349 # Raise error if there is already a running Spark context 
 --> 350 raise ValueError(
 351 "Cannot run multiple SparkContexts at once; " 
 352 "existing SparkContext(app=%s, master=%s)" 

 ValueError : Cannot run multiple SparkContexts at once; existing SparkContext(app=Databricks Shell, master=local[8]) created by __init__ at /databricks/python_shell/scripts/PythonShellImpl.py:1569

In [0]:
# Q2
book_43 = sc.textFile('./FileStore/shared_uploads/wongk711@hawaii.edu/data/43.txt')

str(type(book_43)) == "<class 'pyspark.rdd.RDD'>"

Out[4]: True

In [0]:
# Q3
book_43.count()

Out[5]: 2935

In [0]:
# Q4
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()

test_line = "This is an example of that contains 234 and a dash-containing number"
clean_split_line(test_line)

Out[6]: ['THIS',
 'IS',
 'AN',
 'EXAMPLE',
 'OF',
 'THAT',
 'CONTAINS',
 'AND',
 'A',
 'DASH',
 'CONTAINING',
 'NUMBER']

In [0]:
# Q5
words = book_43.flatMap(clean_split_line)
words.count()

Out[7]: 29116

In [0]:
# Q6
# converts to upper but still functions for couinting purposes
unique = book_43.flatMap(clean_split_line).distinct().count()
unique

Out[8]: 4296

In [0]:
# Q7
book_43_counts = book_43.flatMap(clean_split_line).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y)
book_43_counts.take(1)

Out[9]: [('GUTENBERG', 98)]

In [0]:
# Q8
swapped = book_43_counts.map(lambda x: (x[1], x[0]))
swapped.sortByKey(False).take(20)

Out[10]: [(1807, 'THE'),
 (1068, 'OF'),
 (1043, 'AND'),
 (726, 'TO'),
 (686, 'A'),
 (646, 'I'),
 (485, 'IN'),
 (471, 'WAS'),
 (392, 'THAT'),
 (384, 'HE'),
 (378, 'IT'),
 (312, 'YOU'),
 (308, 'MY'),
 (301, 'WITH'),
 (285, 'HIS'),
 (244, 'HAD'),
 (203, 'AS'),
 (202, 'FOR'),
 (195, 'THIS'),
 (193, 'BUT')]

In [0]:
# Q9
from sklearn.feature_extraction import _stop_words
stop_words = list(_stop_words.ENGLISH_STOP_WORDS)
stop_words_upper = [s.upper() for s in stop_words]
len(stop_words_upper), stop_words_upper[:10]

Out[11]: (318,
 ['WAS',
 'HOW',
 'FOR',
 'COULD',
 'PLEASE',
 'THEN',
 'WHEREUPON',
 'SO',
 'AS',
 'ANY'])

In [0]:
# Q10
book_43_counts_filtered = book_43_counts.filter(lambda x: all(val not in x for val in stop_words_upper))
book_43_counts_filtered.collect()[:5]

Out[12]: [('GUTENBERG', 98), ('DR', 49), ('JEKYLL', 105), ('MR', 128), ('ROBERT', 3)]

In [0]:
# Q11
stop_words_rdd = sc.parallelize(stop_words_upper)
words_in_both = book_43_counts.keys().intersection(stop_words_rdd).count()

assert(book_43_counts.count() - words_in_both == book_43_counts_filtered.count())

print(f"book_43_counts: \t\t\t {book_43_counts.count()}")
print(f"book_43_counts intersect stop_words_rdd:  {words_in_both}")
print(f"book_43_counts_filtered: \t\t {book_43_counts_filtered.count()}")

book_43_counts: 4296
book_43_counts intersect stop_words_rdd: 262
book_43_counts_filtered: 4034

In [0]:
# Q12
def process_RDD(text_file):
    book = sc.textFile(text_file)
    book_counts = book.flatMap(clean_split_line).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y)
    stop_words = list(_stop_words.ENGLISH_STOP_WORDS)
    stop_words_upper = [s.upper() for s in stop_words]
    return book_counts.filter(lambda x: all(val not in x for val in stop_words)).collect()

process_RDD('./FileStore/shared_uploads/wongk711@hawaii.edu/data/43.txt')[:5]

Out[14]: [('GUTENBERG', 98), ('OF', 1068), ('DR', 49), ('JEKYLL', 105), ('MR', 128)]

In [0]:
# Q13
book_84_counts_filtered = process_RDD('./FileStore/shared_uploads/wongk711@hawaii.edu/data/84.txt')
book_398_counts_filtered = process_RDD('./FileStore/shared_uploads/wongk711@hawaii.edu/data/398.txt')
book_3296_counts_filtered = process_RDD('./FileStore/shared_uploads/wongk711@hawaii.edu/data/3296.txt')

print(f"book_84: {len(book_84_counts_filtered)}")
print(f"book_398: {len(book_398_counts_filtered)}")
print(f"book_3296: {len(book_3296_counts_filtered)}")

book_84: 7286
book_398: 2669
book_3296: 7572

In [0]:
# Q14
# we'll use 43 later
book_43_counts_filtered = process_RDD('./FileStore/shared_uploads/wongk711@hawaii.edu/data/43.txt')
book_rdd = {43: sc.parallelize(book_43_counts_filtered), 398: sc.parallelize(book_398_counts_filtered), 84: sc.parallelize(book_84_counts_filtered), 3296: sc.parallelize(book_3296_counts_filtered)}

# compare 398 to 3296    
print(f"compare book_398 to book_3296: {book_rdd[398].keys().intersection(book_rdd[3296].keys()).count()}")
# compare 398 to 84
print(f"compare book_398 to book_84: {book_rdd[398].keys().intersection(book_rdd[84].keys()).count()}")
# compare 3296 to 84
print(f"compare book_3296 to book_84: {book_rdd[3296].keys().intersection(book_rdd[84].keys()).count()}")

compare book_398 to book_3296: 2035
compare book_398 to book_84: 1925
compare book_3296 to book_84: 3864

In [0]:
# Q15

# I don't think comparing shared distinct words is a good idea for a distance metric because the values are out of proportion/context. 
# For example books 84 and 3296 have over 7200 words where as 398 only has ~2600 words so the comparisons aren't valid.
# A slightly improved version might be to compare the top ~1000 most frequent words. 
# this would also help eliminate a case where the intersection is counted if a highly frequented word in one book matches a word mentioned only once in another.

In [0]:
# PART 2

In [0]:
# Q16
import numpy as np

# keys are distinct words so we don't have to worry about duplicate words
joined = book_rdd[43].join(book_rdd[84]).join(book_rdd[398]).join(book_rdd[3296]).collect()
counts_matrix = np.zeros([4,len(joined)])
counts_matrix[0,:] = [joined[i][1][0][0][0] for i in range(len(joined))]
counts_matrix[1,:] = [joined[i][1][0][0][1] for i in range(len(joined))]
counts_matrix[2,:] = [joined[i][1][0][1] for i in range(len(joined))]
counts_matrix[3,:] = [joined[i][1][1] for i in range(len(joined))]
counts_matrix

Out[19]: array([[16., 2., 1., ..., 1., 4., 1.],
 [21., 14., 2., ..., 19., 4., 2.],
 [10., 3., 31., ..., 27., 4., 1.],
 [16., 4., 5., ..., 5., 4., 1.]])

In [0]:
# temp
print(joined.collect()[:3])
print(joined.collect()[1][1][0][0][0]) 
print(joined.collect()[1][1][0][0][1])
print(joined.collect()[1][1][0][1])
print(joined.collect()[1][1][1])

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-4109961982314514> in <module> 
 1 # temp 
 ----> 2 print ( joined . collect ( ) [ : 3 ] ) 
 3 print ( joined . collect ( ) [ 1 ] [ 1 ] [ 0 ] [ 0 ] [ 0 ] ) 
 4 print ( joined . collect ( ) [ 1 ] [ 1 ] [ 0 ] [ 0 ] [ 1 ] ) 
 5 print ( joined . collect ( ) [ 1 ] [ 1 ] [ 0 ] [ 1 ] ) 

 AttributeError : 'list' object has no attribute 'collect'

In [0]:
# Q17
from scipy.spatial.distance import euclidean 

# counts_matrix 0, 1, 2, 3 represent books 43, 84, 398, 3296 respectively

# compare 398 and 3296
print(f"Euclidean Distance between 398 and 3296: {euclidean(counts_matrix[2], counts_matrix[3]) }")
# compare 398 and 84
print(f"Euclidean Distance between 398 and 84: {euclidean(counts_matrix[2], counts_matrix[1]) }")

# as we can see below, while the distances are least proportional and show relevant value, they do not align with what we know.
# for instance we know books 398 and 3296 are both about the same topic of religion so they should be closer than books 398 and 84,
# however we see the opposite so I don't believe this particular method involving euclidean distance is viable.

Euclidean Distance between 398 and 3296: 7269.934593928614
Euclidean Distance between 398 and 84: 4983.69431245537

In [0]:
# Q18
# This method takes only the words which books share in common into account, 
# therefore we are potentially overlooking words that are the main topic of one book and are not mentioned in the other.
# for example if book A mentions "dog" 500 times but book B doesn't mention it once, we don't count this word,
# despite it's potential to greatly affect the "distance" between the two books.
# one way of implementing this might be to (for our example) say book B mentions dog 0 times that way we can count the added 500 word units of this dimension to the distance metric

In [0]:
# PART 3

In [0]:
# Q19
from pyspark.sql import SparkSession
session = SparkSession(sc)
flight_info = session.read.csv('/FileStore/shared_uploads/wongk711@hawaii.edu/data/flight_info.csv', header=True)
flight_info.count()

Out[24]: 450017

In [0]:
# Q20
flight_info.createOrReplaceTempView('flight_info')
session.sql("""SELECT DISTINCT UniqueCarrier FROM flight_info""").show()

+-------------+
UniqueCarrier|
+-------------+
 NK|
 AA|
 EV|
 B6|
 DL|
 F9|
 HA|
 UA|
 OO|
 WN|
 AS|
 VX|
+-------------+

In [0]:
# Q21
session.sql("""
SELECT UniqueCarrier, count(*) AS FlightsDelayed
FROM flight_info 
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier
ORDER BY FlightsDelayed DESC
""").show()

+-------------+--------------+
UniqueCarrier|FlightsDelayed|
+-------------+--------------+
 WN| 54557|
 DL| 28962|
 AA| 26291|
 UA| 19594|
 OO| 17924|
 EV| 12340|
 B6| 10406|
 AS| 4966|
 NK| 4435|
 F9| 3181|
 VX| 2871|
 HA| 2265|
+-------------+--------------+

In [0]:
# Q22
airlines_info = session.read.csv('/FileStore/shared_uploads/wongk711@hawaii.edu/data/airlines.csv')
airlines_info.createOrReplaceTempView('airlines_info')
session.sql("""
SELECT flight_info.UniqueCarrier, first(airlines_info._c1) AS AirlineName, count(*) AS FlightsDelayed
FROM flight_info
INNER JOIN airlines_info
ON airlines_info._c3 = flight_info.UniqueCarrier
WHERE flight_info.DepTime > flight_info.CRSDepTime
GROUP BY flight_info.UniqueCarrier
ORDER BY FlightsDelayed DESC
""").show()

+-------------+--------------------+--------------+
UniqueCarrier| AirlineName|FlightsDelayed|
+-------------+--------------------+--------------+
 WN| Southwest Airlines| 54557|
 DL| Delta Air Lines| 28962|
 AA| American Airlines| 26291|
 UA| United Airlines| 19594|
 OO| SkyWest| 17924|
 EV|Atlantic Southeas...| 12340|
 B6| JetBlue Airways| 10406|
 AS| Alaska Airlines| 4966|
 NK| Spirit Airlines| 4435|
 F9| Frontier Airlines| 3181|
 VX| Virgin America| 2871|
 HA| Hawaiian Airlines| 2265|
+-------------+--------------------+--------------+

In [0]:
# Q23
result = session.sql("""
SELECT UniqueCarrier, DayOfWeek, count(*) AS FlightsDelayed
FROM flight_info
WHERE DepTime > CRSDepTime
GROUP BY UniqueCarrier, DayOfWeek
ORDER BY UniqueCarrier ASC, DayOfWeek ASC
""")
result.show()

+-------------+---------+--------------+
UniqueCarrier|DayOfWeek|FlightsDelayed|
+-------------+---------+--------------+
 AA| 1| 5117|
 AA| 2| 3688|
 AA| 3| 2941|
 AA| 4| 3575|
 AA| 5| 3525|
 AA| 6| 2800|
 AA| 7| 4645|
 AS| 1| 872|
 AS| 2| 669|
 AS| 3| 618|
 AS| 4| 728|
 AS| 5| 630|
 AS| 6| 517|
 AS| 7| 932|
 B6| 1| 1892|
 B6| 2| 1787|
 B6| 3| 1253|
 B6| 4| 1254|
 B6| 5| 1309|
 B6| 6| 1027|
+-------------+---------+--------------+
only showing top 20 rows

In [0]:
%%time
# Q24 
flight_count = session.sql("""
SELECT UniqueCarrier, DayOfWeek, count(*) AS Flights
FROM flight_info
GROUP BY UniqueCarrier, DayOfWeek
""")
flight_count.createOrReplaceTempView('flight_count')

session.sql("""
SELECT info.UniqueCarrier, info.DayOfWeek, 
    count(info.DepTime - info.CRSDepTime) / first(count.flights) AS NormalizedDelay
FROM flight_info AS info
INNER JOIN flight_count AS count
ON info.UniqueCarrier = count.UniqueCarrier AND info.DayOfWeek = count.DayOfWeek
WHERE info.DepTime > info.CRSDepTime
GROUP BY info.UniqueCarrier, info.DayOfWeek
ORDER BY info.UniqueCarrier ASC, info.DayOfWeek ASC
""").show()


+-------------+---------+-------------------+
UniqueCarrier|DayOfWeek| NormalizedDelay|
+-------------+---------+-------------------+
 AA| 1| 0.4206329634196465|
 AA| 2|0.31950099627479855|
 AA| 3|0.30285243538255585|
 AA| 4|0.36133009904992924|
 AA| 5|0.35448511665325827|
 AA| 6| 0.3415883859948762|
 AA| 7| 0.3977564651481418|
 AS| 1| 0.3553382233088835|
 AS| 2|0.29099608525445847|
 AS| 3| 0.3333333333333333|
 AS| 4| 0.3731419784725782|
 AS| 5|0.32142857142857145|
 AS| 6| 0.2875417130144605|
 AS| 7|0.38914405010438413|
 B6| 1|0.45811138014527847|
 B6| 2|0.44899497487437184|
 B6| 3|0.39601769911504425|
 B6| 4|0.38278388278388276|
 B6| 5|0.40055079559363527|
 B6| 6| 0.3583391486392184|
+-------------+---------+-------------------+
only showing top 20 rows

CPU times: user 24 ms, sys: 8.02 ms, total: 32.1 ms
Wall time: 7.28 s

In [0]:
# Q25
times = [8.08, 8.78, 8.13]
sum(times) / len(times)

Out[41]: 8.33

In [0]:
%%time
# Q26
result.join(flight_count,
            [result.UniqueCarrier == flight_count.UniqueCarrier, 
             result.DayOfWeek == flight_count.DayOfWeek]
           ).select(
    result.UniqueCarrier, result.DayOfWeek, result.FlightsDelayed / flight_count.Flights
).orderBy(['UniqueCarrier', 'DayOfWeek']).show()

+-------------+---------+--------------------------+
UniqueCarrier|DayOfWeek|(FlightsDelayed / Flights)|
+-------------+---------+--------------------------+
 AA| 1| 0.4206329634196465|
 AA| 2| 0.31950099627479855|
 AA| 3| 0.30285243538255585|
 AA| 4| 0.36133009904992924|
 AA| 5| 0.35448511665325827|
 AA| 6| 0.3415883859948762|
 AA| 7| 0.3977564651481418|
 AS| 1| 0.3553382233088835|
 AS| 2| 0.29099608525445847|
 AS| 3| 0.3333333333333333|
 AS| 4| 0.3731419784725782|
 AS| 5| 0.32142857142857145|
 AS| 6| 0.2875417130144605|
 AS| 7| 0.38914405010438413|
 B6| 1| 0.45811138014527847|
 B6| 2| 0.44899497487437184|
 B6| 3| 0.39601769911504425|
 B6| 4| 0.38278388278388276|
 B6| 5| 0.40055079559363527|
 B6| 6| 0.3583391486392184|
+-------------+---------+--------------------------+
only showing top 20 rows

CPU times: user 17.6 ms, sys: 11 ms, total: 28.7 ms
Wall time: 6.8 s

In [0]:
# Run time above is 6.8 seconds which is ~1.5 seconds faster than 8.33

In [0]:
# Q27
import pandas as pd

data = session.sql("""
SELECT DepTime - CRSDepTime AS x, ArrTime - CRSArrTime AS y
FROM flight_info
WHERE DepTime - CRSDepTime > 0
""").toPandas().dropna()
data

Out[59]:

,x,y
0,37.0,73.0
1,13.0,2.0
2,17.0,65.0
3,7.0,3.0
4,9.0,-6.0
...,...,...
166920,14.0,6.0
166921,61.0,3.0
166922,48.0,100.0
166923,114.0,122.0


In [0]:
from scipy.stats import linregress

slope, intercept, r_value, p_value, std_err = linregress(data['x'], data['y'])
print(f"slope: {slope}")
print(f"y-intercept: {intercept}")
print(f"p-value: {p_value}")
print(f"standard error: {std_err}")
print(f"r-value (correlation coefficient): {r_value}")
print(f"r-value^2: {r_value**2}")

slope: -0.2174466955626679
y-intercept: -6.381676895789772
p-value: 4.0363034436612525e-92
standard error: 0.010675077132589757
r-value (correlation coefficient): -0.04986210877304325
r-value^2: 0.0024862298912947966

In [0]:
# As we can see, r^2 value is very close to 0 indicating that departure delay time has no correlation with arrival delay time. 
# In other words, if your flight is delayed by 30 minutes, there is no evidence to support the statement that you will arrive 30 minutes later than scheduled.